In [11]:
import pandas as pd

In [12]:
"""
Loads in raw score file
Expecting the schema 
Name	Date	GF	GA	Team

Calcuates result, margies and goal difference
Does some prep work for Form and Win

Obviously names need to be consistant
"""

raw_data = pd.read_csv("../data/results_v2.csv")
raw_data["GD"] = raw_data["GF"] - raw_data["GA"]
raw_data.loc[(raw_data['GD'] == 0), 'GamePoints'] = 0.5
raw_data.loc[(raw_data['GD'] < 0), 'GamePoints'] = -1
raw_data.loc[(raw_data['GD'] > 0), 'GamePoints'] = 1
raw_data.loc[(raw_data['GD'] > 6), 'GamePoints'] = 1.5
raw_data.loc[(raw_data['GD'] < -6), 'GamePoints'] = -1.5
raw_data.loc[(raw_data['GamePoints'] >= 1), 'W'] = 1
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'D'] = 1
raw_data.loc[(raw_data['GamePoints'] <= -1), 'L'] = 1
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'MW'] = 1
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'ML'] = 1

raw_data.loc[(raw_data['GamePoints'] >= 1), 'Result'] = '🔼'
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'Result'] = '➖'
raw_data.loc[(raw_data['GamePoints'] <= -1), 'Result'] = '🔽'
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'Result'] = '⏫'
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'Result'] = '⏬'

raw_data = raw_data[raw_data["Name"].str.contains("\*")==False] # Removes Ringers

raw_data = raw_data.fillna(0)
pd.set_option('display.float_format', '{:.1f}'.format)
raw_data = raw_data.astype({'W':'int','D':'int','L':'int','MW':'int','ML':'int'})

raw_data['Date'] = pd.to_datetime(raw_data['Date'],dayfirst = True)
gameweek_int = raw_data.nunique().Date
gameweek = "Gameweek " + str(gameweek_int)

#display(raw_data)

In [13]:
"""
Creates the Form Column
"""
def form_column(raw_data):
    raw_data['game_rank'] = raw_data.sort_values("Date").groupby("Name").Date.rank(ascending=False)
    raw_data_form = raw_data.loc[raw_data['game_rank'] <= 5]
    form = raw_data_form.groupby(['Name'], as_index=False).agg({'Result': ' '.join})
    return form

form = form_column(raw_data)

#display(form)

In [14]:
"""
Creates Matches Played (MP), Partipation (Par%), Pts and Win% (W%)

It also aggreates all the raw data to create an up to date table
"""
def get_totals(raw_data):
    played = raw_data.groupby('Name').count()
    played = pd.DataFrame(played["Date"]).rename(columns={"Date":"MP"})
    data = raw_data.groupby('Name').sum(numeric_only=True).sort_values(by=['GamePoints'],ascending=False).join(played).rename(columns={"GamePoints":"Pts"})

    data["Par%"] = data["MP"].div(gameweek_int)*100
    data["W%"] = data["W"]/data["MP"]*100
    data = data.astype({'Par%':'int','W%':'int'})
    return data

totals = get_totals(raw_data)



#display(data)


In [15]:
"""
Join tables 
Drops unwanted columns 
Puts columns in the correct order
"""
table = totals.merge(form,on='Name').rename(columns={"Result":"Form"})
table.index = table.Name
table = table[['MP', 'W', 'D', 'L','MW', 'ML', 'GF', 'GA','GD','Par%','W%','Pts','Form']]
table.index.names = [gameweek]
table

,MP,W,D,L,MW,ML,GF,GA,GD,Par%,W%,Pts,Form
Gameweek 12,,,,,,,,,,,,,
Wilf,9,7,0,2,2,2,142,126,16,75,77,5.0,🔼 ⏬ 🔼 ⏬ 🔼
Calz,8,5,0,3,3,0,130,102,28,66,62,3.5,🔼 ⏫ 🔽 ⏫ 🔽
Schurrle,7,5,0,2,2,1,119,103,16,58,71,3.5,🔼 ⏫ ⏬ 🔽 🔼
Virlo,4,3,0,1,2,0,70,55,15,33,75,3.0,⏫ ⏫ 🔼 🔽
Keysey,3,2,0,1,0,0,47,43,4,25,66,1.0,🔼 🔽 🔼
Doug,6,3,0,3,2,1,87,79,8,50,50,0.5,🔽 ⏫ 🔼 ⏬ 🔽
Zola,4,2,0,2,1,0,60,60,0,33,50,0.5,⏫ 🔼 🔽 🔽
Mate,11,5,0,6,2,2,169,159,10,91,45,-1.0,⏫ ⏫ 🔽 🔼 🔼
Peggy,3,1,0,2,0,0,44,47,-3,25,33,-1.0,🔽 🔽 🔼


In [16]:
"""
TO DO

Add Graphics
Tracking player pairings
Tracking week by week table position

"""


'\nTO DO\n\nAdd Graphics\nTracking player pairings\nTracking week by week table position\n\n'

In [17]:
display(raw_data)

,Name,Date,GF,GA,Team,GD,GamePoints,W,D,L,MW,ML,Result,game_rank
0,Calz,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,8.0
1,Helmet,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,8.0
2,Kim,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,11.0
3,Mate,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,11.0
4,Peggy,2023-03-02,16,17,DARKS,-1,-1.0,0,0,1,0,0,🔽,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Calz,2023-05-11,13,19,DARKS,-6,-1.0,0,0,1,0,0,🔽,1.0
116,Wilf,2023-05-11,19,13,BLANCOS,6,1.0,1,0,0,0,0,🔼,1.0
117,Mate,2023-05-11,19,13,BLANCOS,6,1.0,1,0,0,0,0,🔼,1.0
118,Kim,2023-05-11,19,13,BLANCOS,6,1.0,1,0,0,0,0,🔼,1.0


In [18]:
def count_same_team(df, name_col, date_col, team_col):
    count = 0
    for idx, row in df.iterrows():
        name = row[name_col]
        date = row[date_col]
        team = row[team_col]
        same_team_df = df[(df[team_col] == team) & (df[date_col] == date)]
        same_team_names = same_team_df[name_col].unique()
        if len(same_team_names) > 1 and name in same_team_names:
            print(same_team_names)
    return count

In [19]:
count_same_team(raw_data, 'Name', 'Date', 'Team')

['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Wilf']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Wilf']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Wilf']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Wilf']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Wilf']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Calz' 'Kim' 'Virlo' 'Wilf' 'Doug']
['Calz' 'Kim' 'Virlo' 'Wilf' 'Doug']
['Calz' 'Kim' 'Virlo' 'Wilf' 'Doug']
['Calz' 'Kim' 'Virlo' 'Wilf' 'Doug']
['Calz' 'Kim' 'Virlo' 'Wilf' 'Doug']
['Gilet' 'Schurrle' 'Virlo' 'Wilf']
['Gilet' 'Schurrle' 'Virlo' 'Wilf']
['Gilet' 'Schurrle' 'Virlo' 'Wilf']
['Gilet' 'Schurrle' 'Virlo' 'Wilf']
['Kim' 'Mate' 'Noel' 'Reaper']
['Kim

0